# Cho et. al. 's cube root algorithm when p=1 (mod3)

In [1]:
import timeit
from random import *
import math

In [2]:
#Find prime p which is 100 bits
#In order to find such p, I use Miller-Rabin primality test.
#In this cube root algorithm, we use p as 1 mod 3, so I add this condition in the end.

# Python3 program Miller-Rabin primality test
import random


# This function is called for all k trials. 
# It returns false if n is composite and returns false if n is probably prime.
def miillerTest(d, n):

    a = 2 + random.randint(1, n - 4)

    x = pow(a, d, n)

    if (x == 1 or x == n - 1):
        return True

    while (d != n - 1):
        x = (x * x) % n
        d *= 2

        if (x == 1):
            return False
        if (x == n - 1):
            return True

    return False;

# It returns false if n is composite and returns true if n is probably prime. 
# k is an input parameter that determines accuracy level. Higher value of k indicates more accuracy.
def isPrime( n, k):

    if (n <= 1 or n == 4):
        return False
    if (n <= 3):
        return True

    d = n - 1
    while (d % 2 == 0):
        d //= 2
    
    for i in range(k):
        if (miillerTest(d, n) == False):
            return False

    return True


# Driver Code
k = 4 # Number of iterations


while 1:
    p = random.randint(pow(2,100),pow(2,101))
    if (isPrime(p, k) and (p%3==1)):
        print("Our sample 100 bit random prime number p =",p)
        break

Our sample 100 bit random prime number p = 1893007777900274201869190125081


In [3]:
#squre root 구하는 알고리즘 추가


def modular_sqrt(a, p):

    def legendre_symbol(a, p):
        """ Compute the Legendre symbol a|p using
            Euler's criterion. p is a prime, a is
            relatively prime to p (if p divides
            a, then a|p = 0)
            Returns 1 if a has a square root modulo
            p, -1 otherwise.
        """
        ls = pow(a, (p - 1) // 2, p)
        return -1 if ls == p - 1 else ls

    """ Find a quadratic residue (mod p) of 'a'. p
        must be an odd prime.
        Solve the congruence of the form:
            x^2 = a (mod p)
        And returns x. Note that p - x is also a root.
        0 is returned is no square root exists for
        these a and p.
        The Tonelli-Shanks algorithm is used (except
        for some simple cases in which the solution
        is known from an identity). This algorithm
        runs in polynomial time (unless the
        generalized Riemann hypothesis is false).
    """
    # Simple cases
    #
    if legendre_symbol(a, p) != 1:
        return 0
    elif a == 0:
        return 0
    elif p == 2:
        return p
    elif p % 4 == 3:
        return pow(a, (p + 1) // 4, p)

    # Partition p-1 to s * 2^e for an odd s (i.e.
    # reduce all the powers of 2 from p-1)
    #
    s = p - 1
    e = 0
    while s % 2 == 0:
        s //= 2
        e += 1

    # Find some 'n' with a legendre symbol n|p = -1.
    # Shouldn't take long.
    #
    n = 2
    while legendre_symbol(n, p) != -1:
        n += 1

    # Here be dragons!
    # Read the paper "Square roots from 1; 24, 51,
    # 10 to Dan Shanks" by Ezra Brown for more
    # information
    #

    # x is a guess of the square root that gets better
    # with each iteration.
    # b is the "fudge factor" - by how much we're off
    # with the guess. The invariant x^2 = ab (mod p)
    # is maintained throughout the loop.
    # g is used for successive powers of n to update
    # both a and b
    # r is the exponent - decreases with each update
    #
    x = pow(a, (s + 1) // 2, p)
    b = pow(a, s, p)
    g = pow(n, s, p)
    r = e

    while True:
        t = b
        m = 0
        for m in range(r):
            if t == 1:
                break
            t = pow(t, 2, p)

        if m == 0:
            return x

        gs = pow(g, 2 ** (r - m - 1), p)
        g = (gs * gs) % p
        x = (x * gs) % p
        b = (b * g) % p
        r = m

In [4]:
def Precompute_Cho_mod3(R,p):
    t = 1
    Q = (R*(t**3)+3)%p
    D = (-(4*((Q-3)**3)+27*((Q-3)**2)))%p
    A=(108*(Q-3))%p
    B=(12*(Q-3))%p
    sqr=modular_sqrt(A**2+B**3, p)
    alpha =(A+sqr)%p


          
    while (pow(int(D),int((p-1)//2),p)!=1) or (pow(int(alpha),int((p-1)//3),p)==1) :

        t = t+1
        Q = (R*(t**3)+3)%p  
        D = (-(4*((Q-3)**3)+27*((Q-3)**2)))%p
        A=(108*(Q-3))%p
        B=(12*(Q-3))%p
        sqr=modular_sqrt(A**2+B**3,p)
        alpha = (A+sqr)%p
        

    return(Q,t)

#int(math.sqrt(-3*D))값은 실수에서 계산하여 소수점 자리를 버리고 정수로 표현, 즉 modulo p로 square root를 찾는 것이 아님.
#그래서 square root 찾는 알고리즘 이용하여 계산하는 것으로 변경
#그럼에도 reducible polynomial이 간혹 output됨

In [5]:
def w2n(W,P,Q,R,p):
    x=(W[0]*W[0]+R*(2*W[1]+P*W[2])*W[2])%p
    y=(2*(W[0]-Q*W[2])*W[1]+(R-P*Q)*W[2]*W[2])%p
    z=(2*W[0]*W[2]+W[1]*W[1]+P*(2*W[1]*W[2]+P*W[2]*W[2])-Q*W[2]*W[2])%p
    return(int(x),int(y),int(z))

def wm1(W,P,Q,R,p):
    x=(R*W[2])%p
    y=(W[0]-Q*W[2])%p
    z=(W[1]+P*W[2])%p
    return(int(x),int(y),int(z))

def wmn(W,V,P,Q,R,p):
    x=(W[0]*V[0]+R*(W[1]*V[2]+V[1]*W[2])+P*R*W[2]*V[2])%p
    y=(W[0]*V[1]+V[0]*W[1]-Q*(W[1]*V[2]+V[1]*W[2])+(R-P*Q)*W[2]*V[2])%p
    z=(W[0]*V[2]+W[1]*V[1]+V[0]*W[2]+P*(W[1]*V[2]+V[1]*W[2])+(P*P-Q)*W[2]*V[2])%p
    return(int(x),int(y),int(z))

def wpn(W,V,P,Q,R,p):
    x=(W[0]+V[0]*W[1]+(V[0]*V[0]+(2*V[1]+P*V[2])*R*V[2])*W[2])%p
    y=(W[1]*V[1]+(2*(V[0]-Q*V[2])*V[1]+(R-P*Q)*V[2]*V[2])*W[2])%p
    z=(V[2]*W[1]+(2*(V[0]+P*V[1])*V[2]+V[1]*V[1]+(P*P-Q)*V[2]*V[2])*W[2])%p
    return(int(x),int(y),int(z))


In [6]:
def Cho_mod3(P,Q,R,p,t):
    m=bin(int((p-1)//3))
    l=len(m)
    x=0;y=1;z=0      
    W=(x,y,z)      
    for i in range(3,l):
        W=w2n(W,P,Q,R,p)

        if(m[i]=='1'):
                  W=wm1(W,P,Q,R,p)  
                
    w2s=(w2n(W,P,Q,R,p)[0],w2n(W,P,Q,R,p)[1],w2n(W,P,Q,R,p)[2])
    w2s1=(wm1(w2s,P,Q,R,p)[0],wm1(w2s,P,Q,R,p)[1],wm1(w2s,P,Q,R,p)[2])      
    w3s=(wmn(W,w2s,P,Q,R,p)[0],wmn(W,w2s,P,Q,R,p)[1],wmn(W,w2s,P,Q,R,p)[2])
    wp=(wm1(w3s,P,Q,R,p)[0],wm1(w3s,P,Q,R,p)[1],wm1(w3s,P,Q,R,p)[2])
    wps=(wpn(W,wp,P,Q,R,p)[0],wpn(W,wp,P,Q,R,p)[1],wpn(W,wp,P,Q,R,p)[2])
    wq=(wmn(wps,w2s1,P,Q,R,p)[0],wmn(wps,w2s1,P,Q,R,p)[1],wmn(wps,w2s1,P,Q,R,p)[2])
    
    print(wq[1]%p,wq[2]%p)
    return(int(wq[0]*pow(t,-1,p))%p)      


In [7]:
#Main 
a=randrange(1,p)
R=int((a*a*a)%p)
Q, t = Precompute_Cho_mod3(R,p)
P=3

cube_root=Cho_mod3(P,Q,R,p,t)
print('cube root of R:',cube_root)
print('check:',int((cube_root*cube_root*cube_root)%p)==int(R))

0 0
cube root of R: 410160648579792177729140066450
check: True


In [9]:
time=0
for i in range(10) :
    print("Experiment : ",i+1)
    a=randrange(1,p)
    R=int((a*a*a)%p)
    Q, t = Precompute_Cho_mod3(R,p)
    P=3
    print("sample p and cubic residue R, respectively :",p,R)
    cube_root=Cho_mod3(P,Q,R,p,t)
    print('check:',int((cube_root**3)%p)==R)
    t1 = timeit.timeit(str(Cho_mod3(P,Q,R,p,t)),setup='pass', number=1)
    print("cpu time for executing Body part of Cho's algorithm (mod3):", t1)
    print("=============================================================")
    
    time = time+t1

Experiment :  1
sample p and cubic residue R, respectively : 1893007777900274201869190125081 914555144220280279492575025284
0 0
check: False
0 0
cpu time for executing Body part of Cho's algorithm (mod3): 9.000000318337698e-07
Experiment :  2
sample p and cubic residue R, respectively : 1893007777900274201869190125081 1279091938697997360606491613245
1010146305056525542306898588838 1040036267577207215738212946173
check: False
1010146305056525542306898588838 1040036267577207215738212946173
cpu time for executing Body part of Cho's algorithm (mod3): 8.000000661922968e-07
Experiment :  3
sample p and cubic residue R, respectively : 1893007777900274201869190125081 618943617904462321286782963056
805775591798023760109548314855 506396388899436760858416096813
check: False
805775591798023760109548314855 506396388899436760858416096813
cpu time for executing Body part of Cho's algorithm (mod3): 7.999999525054591e-07
Experiment :  4
sample p and cubic residue R, respectively : 189300777790027420186